## K-Nearest Neighbors (KNN) Algorithm

### Principle Intuition
K-Nearest Neighbors (KNN) is a simple, non-parametric, instance-based learning algorithm used for classification and regression tasks. The fundamental idea behind KNN is that given a data point, its class or value can be predicted by looking at the labels of its closest neighbors. KNN assumes that similar points are close to each other in the feature space, which means that nearby points in the training data provide useful information for predicting the label of a new data point.

When a new data point needs to be classified:
1. The algorithm identifies the **k** closest points from the training set.
2. These **k** neighbors "vote" by providing their labels, and the new data point is assigned to the class with the majority vote (for classification) or the average of their values (for regression).

### Distances Used in KNN
The performance of KNN is largely dependent on how distances between data points are measured. Several distance metrics can be used in KNN, with the most common ones being:

1. **Euclidean Distance** (L2 Norm):
   \[
   d(p, q) = \sqrt{\sum_{i=1}^{n}(p_i - q_i)^2}
   \]
   This is the most common distance measure, which calculates the straight-line distance between two points in n-dimensional space.

2. **Manhattan Distance** (L1 Norm):
   \[
   d(p, q) = \sum_{i=1}^{n} |p_i - q_i|
   \]
   This measures the distance between two points as the sum of the absolute differences of their coordinates, often referred to as "city block distance."

3. **Minkowski Distance**:
   \[
   d(p, q) = \left( \sum_{i=1}^{n} |p_i - q_i|^p \right)^{\frac{1}{p}}
   \]
   This is a generalization of both Euclidean (p=2) and Manhattan (p=1) distances. For any value of **p**, this becomes the Minkowski distance.

4. **Cosine Similarity** (used in high-dimensional sparse data):
   \[
   d(p, q) = 1 - \frac{p \cdot q}{\|p\| \|q\|}
   \]
   Cosine similarity measures the angle between two vectors, making it useful for text and other high-dimensional datasets.

### Choosing the Value of k
The **k** in KNN represents the number of nearest neighbors to consider when making a prediction. Choosing the right value of **k** is crucial for balancing bias and variance:

1. **Small k (e.g., k = 1)**:
   - The algorithm becomes sensitive to noise, leading to high variance and overfitting, as it relies on very local information.
   - If a single noisy data point is among the nearest neighbors, the prediction can be incorrect.

2. **Large k**:
   - A large value of **k** smooths out the decision boundary, leading to lower variance but higher bias, which may result in underfitting.
   - The algorithm looks at a broader group of neighbors, which can dilute the influence of local data points that are most similar.

A common approach is to try several values of **k** and use cross-validation to determine the value that results in the best performance on unseen data.

### Overfitting and Underfitting in KNN
1. **Overfitting**:
   - This happens when the KNN model is too complex and captures noise in the training data, leading to poor generalization to new data.
   - Overfitting in KNN occurs when **k** is too small (e.g., **k = 1**), meaning the model becomes too sensitive to the training data, including outliers and noise.

2. **Underfitting**:
   - Underfitting occurs when the model is too simple to capture the underlying patterns in the data.
   - In KNN, this happens when **k** is too large, and the algorithm averages over too many neighbors, which can smooth over important differences between classes or patterns.


In [1]:
import pandas as pd
import numpy as np

In [4]:
class Distance_metric:
    
    def __init__(self):
        pass
    
    # Euclidean distance
    def euclidean_distance(self, vector1, vector2):
        distance = 0
        for i in range(0, len(vector1)):
            distance += (vector1[i] - vector2[i]) ** 2
        return distance ** 0.5
    
    # Manhattan distance
    def manhattan_distance(self, vector1, vector2):
        distance = 0
        for i in range(0, len(vector1)):
            distance += abs(vector1[i] - vector2[i])
        return distance
    
    # Minkowski distance
    def minkowski_distance(self, vector1, vector2, p=2):
        distance = 0
        for i in range(0, len(vector1)):
            distance += abs(vector1[i] - vector2[i]) ** p
        return distance ** (1/p)
    
    # Hamming distance
    def hamming_distance(self, vector1, vector2):
        distance = 0
        for i in range(0, len(vector1)):
            if vector1[i] != vector2[i]:
                distance += 1
        return distance

In [5]:
class KNearestNeighbour:
    
    def __init__(self, k, metric='euclidean_distance'):
        self.k = k
        self.metric = metric
        
    # Lazy Learning model, Nothing happens in training
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    # Finding all the neighbors of a test row
    def KNeighbours(self, test_row, Distance_metric):
        cal_dist = Distance_metric()
        distances = []
        
        for i, row in enumerate(self.X_train):
            if self.metric == 'euclidean_distance':
                distances.append([row, cal_dist.euclidean_distance(test_row, row), self.y_train[i]])
            elif self.metric == 'manhattan_distance':
                distances.append([row, cal_dist.manhattan_distance(test_row, row), self.y_train[i]])
            elif self.metric == 'minkowski_distance':
                distances.append([row, cal_dist.minkowski_distance(test_row, row), self.y_train[i]])
            elif self.metric == 'hamming_distance':
                distances.append([row, cal_dist.hamming_distance(test_row, row), self.y_train[i]])
        
        # Sort by distance and return the k nearest neighbors
        distances.sort(key=lambda x: x[1])
        return distances[:self.k]
    
    # Inference of the X_test by using K neighbor majority count
    def predict(self, X_test, Distance_metric):
        predictions = []
        
        for row in X_test:
            neighbors = self.KNeighbours(row, Distance_metric)
            output_labels = [label[-1] for label in neighbors]
            pred = max(set(output_labels), key=output_labels.count)
            predictions.append(pred)
            
        return predictions
        